In [1]:
import re  # For preprocessing
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import pandas as pd
import os
os.listdir()
print(os.listdir("Data/"))#Contents of the Data folder

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

['cz_item_category', 'item_content_cz', 'item_content_filtered (with_duplicates).csv', 'item_content_filtered_no_duplicates.csv', 'item_content_viewed_sid.csv', 'item_cz', 'item_list.csv', 'list_category', 'material_attributes_cz', 'product_views_cz', 'product_views_cz_filtered.csv', 'product_views_unified.csv', 'sentences.csv']


In [2]:
dataFile = "Data/product_views_unified.csv"
number_rows = 30000000
data_2020 = pd.read_csv(dataFile,nrows=number_rows)
print(data_2020)

         user_id_anon    product_id
0            14723304  100011803451
1            14723304  100011612041
2            14723304  100016624290
3            14723304       1093107
4            14723304  100016287573
...               ...           ...
2999995      57137615       1489324
2999996      57137615       1285712
2999997      57137615       1489324
2999998      57137615       1399946
2999999      57137615       1400006

[3000000 rows x 2 columns]


In [3]:
data_2020['product_id'] = data_2020.loc[:,'product_id'].map(str)
print(data_2020)

         user_id_anon    product_id
0            14723304  100011803451
1            14723304  100011612041
2            14723304  100016624290
3            14723304       1093107
4            14723304  100016287573
...               ...           ...
2999995      57137615       1489324
2999996      57137615       1285712
2999997      57137615       1489324
2999998      57137615       1399946
2999999      57137615       1400006

[3000000 rows x 2 columns]


In [4]:
sentences = data_2020.groupby('user_id_anon')['product_id'].apply(list)
print(sentences)

user_id_anon
645         [655028, 1338684, 100018643178, 100018644575, ...
788         [100016425290, 452568001, 1086821, 1403084, 14...
871         [551806, 1305893, 1293084, 1389660, 1510064, 1...
1345        [100018205007, 100018206462, 100018206445, 100...
2981        [1480352, 1419661, 1105542, 1419654, 666213, 1...
                                  ...                        
62563472    [1349506, 1349510, 1349506, 1397954, 1490283, ...
62563934    [100016141442, 100016143151, 100016145005, 100...
62565140    [1236035, 100002172473, 645720, 100002786928, ...
62565992    [1339130, 1330960002, 1330960002, 1330960002, ...
62570516    [1327813005, 1327813005, 673184004, 1455333, 1...
Name: product_id, Length: 62897, dtype: object


In [36]:
# most_viewed_element = data_2020["product_id"].value_counts().idxmax()
# print(most_viewed_element)

100024602647


In [5]:
import multiprocessing

from gensim.models import Word2Vec
import numpy as np

cores = multiprocessing.cpu_count() # Count the number of cores in a computer

# w2v_model = Word2Vec(min_count=10,
#                      window=2,
#                      size=300,
#                      sample=6e-5, 
#                      alpha=0.05, 
#                      min_alpha=0.001, 
#                      negative=20,
#                      workers=cores-1)

In [36]:
dataFile = "Data/product_views_unified.csv"
test_rows = 31502086 - number_rows
test_data = pd.read_csv(dataFile,names=["user_id_anon","product_id"],skiprows=number_rows,nrows=test_rows)
test_data['product_id'] = test_data.loc[:,'product_id'].map(str)
test_set = test_data.groupby('user_id_anon')['product_id'].apply(list)
print(test_set)

user_id_anon
2373        [100004358593, 100004358588, 100004360094, 100...
3349        [100019718522, 100019718522, 1478497, 10001971...
4758        [1182921003, 1370068, 557483, 100021006645, 10...
6161        [896698, 100019840359, 896698, 683803, 1184558...
8315        [1366491, 1274773003, 1366491, 776287, 1416626...
                                  ...                        
62552177    [100024815475, 1353749, 2461011, 100024106589,...
62554666    [1455389, 100008723422, 1455389, 1120664, 1395...
62558727    [1503117001, 100018855245, 100004385793, 10002...
62562094    [1498765005, 1498765005, 1498765005, 1489331, ...
62568860    [100019452001, 1144838, 783693, 956190, 125758...
Name: product_id, Length: 21057, dtype: object


In [49]:
import itertools

parm_dict = {'size':(200,250,300),'window':(2,5),'alpha':(0.04,0.035,0.03,0.025,0.02)}

def cust_param_search(parm_dict,sentences,test_set):
    score_best, parm_best = 0,()
   
    size, window, alpha = [tup for k,tup in parm_dict.items()] # Individual parm tuples

    parm_combo = list(itertools.product(size,window, alpha)) # Create all combinations
    score = []
    firstIteration = 1
    test_set_in_vocab = []
    num_history = 5
    for parms in parm_combo:
        s, w, a = parms
        print("Trying parameters:")
        print (parms)
        model = Word2Vec(min_count=10,
                         window=w,
                         size=s,
                         alpha=a,
                         workers=cores-1)
        model.build_vocab(sentences, progress_per=5000)
        model.train(sentences, total_examples=model.corpus_count, epochs=30, report_delay=1)
        model.init_sims(replace=True)
        
        precision = []
        if firstIteration == 1: #upraveni test setu (neobsahuje slova co nejsou ve slovníku + jsou dlouhý alespoň num_history+1)
            print("It is first iteration")
#             print(len(test_set))
            firstIteration = 0
            for i in test_set:
                test = i
                k=0
                while k < len(test):
                        if test[k] not in model.wv.vocab or (test[k] in test[:k]):
                            test.pop(k)
                        else:
                            k=k+1
                if len(test) > num_history:
                    test_set_in_vocab.append(test)
            print("Length of test set:")
            print(len(test_set_in_vocab))
        
        for t in test_set_in_vocab: #SCORE FUNCTION pro nejvíce viděný produkt oproti všem ostatním      
            vector = model.wv.most_similar(positive=t[:num_history],topn=len(t)-num_history)
            vector = [l for (l,n) in vector]
            num = 0
            for l in range(len(vector)):
                if vector[l] in t[num_history:]:
                    num = num + 1
#                         print("true positive")
            precision.append(num / (len(t)-num_history))
        
        score.append(sum(precision)/len(precision)) #MEAN AVERAGE PRECISION scoring function
        print(parms)
        print(score[len(score)-1])

        if score[len(score)-1] > score_best:
            score_best = score[len(score)-1]
            parm_best = parms   
            print("This is best score so far.")
    print("Best score -",score_best, "Best parms - ",parm_best)
    print(score)

In [50]:
cust_param_search(parm_dict,sentences,test_set)

INFO - 13:12:35: collecting all words and their counts
INFO - 13:12:35: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 13:12:35: PROGRESS: at sentence #5000, processed 237915 words, keeping 89055 word types


Trying parameters:
(200, 2, 0.04)


INFO - 13:12:35: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types
INFO - 13:12:35: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 13:12:35: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 13:12:35: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 13:12:36: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 13:12:36: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 13:12:36: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 13:12:36: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 13:12:36: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 13:12:36: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 13:12:36: PROGRE

It is first iteration
Length of test set:
20243


INFO - 13:17:26: collecting all words and their counts
INFO - 13:17:26: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 13:17:26: PROGRESS: at sentence #5000, processed 237915 words, keeping 89055 word types
INFO - 13:17:26: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types


(200, 2, 0.04)
0.01992261699341933
This is best score so far.
Trying parameters:
(200, 2, 0.035)


INFO - 13:17:26: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 13:17:26: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 13:17:26: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 13:17:27: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 13:17:27: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 13:17:27: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 13:17:27: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 13:17:27: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 13:17:27: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 13:17:27: PROGRESS: at sentence #60000, processed 2862554 words, keeping 351953 word types
INFO - 13:17:27: colle

KeyboardInterrupt: 

In [39]:
#S JINOU SCORING FUNKCI

# import itertools

# parm_dict = {'size':(100,200,300),'window':(2,5),'min_count':(5,10),'alpha':(0.03,0.02,0.01)}

# def cust_param_search(parm_dict,sentences,test_set,most_viewed_element):
#     score_best, parm_best = 0,()
   
#     size,window, min_count, alpha = [tup for k,tup in parm_dict.items()] # Individual parm tuples

#     parm_combo = list(itertools.product(size,window, min_count, alpha)) # Create all combinations

#     for parms in parm_combo:
#         s, w, m, a = parms
#         print (parms)
#         model = Word2Vec(min_count=m,
#                          window=w,
#                          size=s,
#                          alpha=a,
#                          workers=cores-1)
#         model.build_vocab(sentences, progress_per=5000)
#         model.train(sentences, total_examples=model.corpus_count, epochs=10, report_delay=1)

#         model.init_sims(replace=True)
#         similarity = []
#         for i in test_set: #SCORE FUNCTION pro nejvíce viděný produkt oproti všem ostatním
#             simil = []
#             for j in i:
#                 if j in model.wv.vocab and j != most_viewed_element:
# #                     print(model.wv.similarity(most_viewed_element,j))
#                     simil.append(model.wv.similarity(most_viewed_element,j))
#             if len(simil)>0:
#                 similarity.append(sum(simil)/len(simil))
        
#         score = sum(similarity)/len(similarity)
#         print(score)

#         if score > score_best:
#             score_best = score
#             parm_best = parms          
#     print("Best score -",score_best, "Best parms - ",parm_best)

In [16]:
# model = Word2Vec(min_count=5,
#                  window=5,
#                  size=200,
#                  alpha=0.025,
#                  workers=cores-1)
# model.build_vocab(sentences, progress_per=5000)
# model.train(sentences, total_examples=model.corpus_count, epochs=20, report_delay=1)

# model.init_sims(replace=True)
# precision = []
# for i in test_set: #SCORE FUNCTION pro nejvíce viděný produkt oproti všem ostatním
#     test = i
#     k=0
#     num_history = 5
#     while k < len(test):
#             if test[k] not in model.wv.vocab:
#                 test.pop(k)
#             else:
#                 k=k+1
#     if len(test) > num_history:        
#         vector = model.wv.most_similar(positive=test[:num_history],topn=len(test)-num_history)
#         vector = [l for (l,n) in vector]
#         num = 0
#         for l in range(len(vector)):
#             if vector[l] in test[num_history:]:
#                 num = num + 1
#                 print("true positive")
#                 print("position " + str(l) + " of " + str(len(vector)) + " positions")
#         precision.append(num / (len(test)-num_history))
# #         print(precision)

# score = sum(precision)/len(precision)
# print(score)

INFO - 16:40:28: collecting all words and their counts
INFO - 16:40:28: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 16:40:28: PROGRESS: at sentence #5000, processed 237915 words, keeping 89055 word types
INFO - 16:40:28: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types
INFO - 16:40:28: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 16:40:28: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 16:40:29: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 16:40:29: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 16:40:29: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 16:40:29: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 16:40:29: PROGRESS: at sentence #45000, processed 2146102 words, keeping 3077

true positive
position 17 of 41 positions
true positive
position 9 of 34 positions
true positive
position 13 of 33 positions
true positive
position 1 of 62 positions
true positive
position 11 of 62 positions
true positive
position 33 of 62 positions
true positive
position 72 of 73 positions
true positive
position 19 of 23 positions
true positive
position 20 of 21 positions
true positive
position 2 of 26 positions
true positive
position 6 of 26 positions
true positive
position 2 of 39 positions
true positive
position 19 of 39 positions
true positive
position 21 of 90 positions
true positive
position 58 of 90 positions
true positive
position 65 of 90 positions
true positive
position 82 of 90 positions
true positive
position 87 of 90 positions
true positive
position 30 of 53 positions
true positive
position 0 of 21 positions
true positive
position 0 of 20 positions
true positive
position 2 of 20 positions
true positive
position 14 of 20 positions
true positive
position 8 of 56 positions
t

In [26]:
# cust_param_search(parm_dict,sentences,test_set,most_viewed_element)

(100, 2, 5, 0.03, 10)


INFO - 19:09:25: collecting all words and their counts
INFO - 19:09:26: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 19:09:42: PROGRESS: at sentence #5000, processed 237915 words, keeping 89055 word types
INFO - 19:09:46: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types
INFO - 19:09:48: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 19:09:48: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 19:09:49: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 19:09:49: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 19:09:50: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 19:09:50: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 19:09:50: PROGRESS: at sentence #45000, processed 2146102 words, keeping 3077

0.016663793677560667
(100, 2, 5, 0.03, 20)


INFO - 19:11:32: PROGRESS: at sentence #10000, processed 478486 words, keeping 138858 word types
INFO - 19:11:32: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 19:11:32: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 19:11:32: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 19:11:32: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 19:11:32: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 19:11:32: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 19:11:33: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 19:11:33: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 19:11:33: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 19:11:33: PROGRE

0.014403753609650636
(100, 2, 5, 0.03, 30)


...

0.012032202309985114
(300, 5, 10, 0.01, 20)


INFO - 22:18:48: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 22:18:48: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 22:18:48: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 22:18:48: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 22:18:48: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 22:18:48: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 22:18:48: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 22:18:48: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 22:18:48: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 22:18:49: PROGRESS: at sentence #60000, processed 2862554 words, keeping 351953 word types
INFO - 22:18:49: colle

0.018515258178757085
(300, 5, 10, 0.01, 30)


INFO - 22:21:45: PROGRESS: at sentence #15000, processed 716817 words, keeping 175553 word types
INFO - 22:21:45: PROGRESS: at sentence #20000, processed 963957 words, keeping 206354 word types
INFO - 22:21:45: PROGRESS: at sentence #25000, processed 1199236 words, keeping 231178 word types
INFO - 22:21:45: PROGRESS: at sentence #30000, processed 1440365 words, keeping 253765 word types
INFO - 22:21:45: PROGRESS: at sentence #35000, processed 1676578 words, keeping 273056 word types
INFO - 22:21:45: PROGRESS: at sentence #40000, processed 1914864 words, keeping 291628 word types
INFO - 22:21:46: PROGRESS: at sentence #45000, processed 2146102 words, keeping 307766 word types
INFO - 22:21:46: PROGRESS: at sentence #50000, processed 2385823 words, keeping 323934 word types
INFO - 22:21:46: PROGRESS: at sentence #55000, processed 2621765 words, keeping 338407 word types
INFO - 22:21:46: PROGRESS: at sentence #60000, processed 2862554 words, keeping 351953 word types
INFO - 22:21:46: colle

0.023133042556102285
Best score - 0.025687542348481238 Best parms -  (200, 5, 10, 0.03, 30)
